<a href="https://colab.research.google.com/github/baciu-andrei/UaicAiCoursesSolutions/blob/main/assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a target="_blank" href="https://colab.research.google.com/github/Tensor-Reloaded/Neural-Networks-Template-2024/blob/main/Lab01/Assignment1.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# **Assignment 1 (10 points)**

## **Solving a linear system in python**

In this homework, you will familiarize yourself with key linear algebra con-
cepts and Python programming by solving a system of linear equations. You
will explore multiple methods for solving such systems, including Cramer’s rule
and matrix inversion. By the end of this assignment, you will have a good un-
derstanding of how to represent and manipulate matrices and vectors in Python.

We begin with the following system of 3 linear equations with 3 unknowns:
$$ 2x + 3y - z = 5 $$
$$ x - y + 4z = 6 $$
$$ 3x + y + 2z = 7 $$

This system can be vectorized in the following form:
$$ A \cdot X = B $$
where:
$$
A = \begin{bmatrix}
2 & 3 & -1 \\
1 & -1 & 4 \\
3 & 1 & 2
\end{bmatrix}, \quad
X = \begin{bmatrix}
x \\
y \\
z
\end{bmatrix}, \quad
B = \begin{bmatrix}
5 \\
6 \\
7
\end{bmatrix}
$$

**Considerations**
- do not use any linear algebra framework such as $numpy$
- use python lists as data structures for matrices and vectors
- experiment with other values for the coefficients and free terms

### **1. Parsing the System of Equations (1 point)**

The first task is to implement a Python script that reads a system of linear equations from a text file and parses it into a matrix $A$ and a vector $B$. You will use the input format described below to extract the coefficients for $A$ and $B$.

**Input File Format**
```text
2x + 3y - z = 5
x - y + 4z = 6
3x + y + 2z = 7
```

Note that the coefficients are always in the order x, y and z and the terms are always space separated

In [ ]:
import pathlib
from google.colab import files

upload = files.upload()

def load_system(path: pathlib.Path) -> tuple[list[list[float]], list[float]]:
    with open(path, 'r') as f:
      lines = f.readlines()
    M = []
    V = []
    def getCoeff(token: str) -> float:
      if token[-1] not in ['x','y','z']:
        return 0.0
      sign = token[:-1]
      if sign == '' or sign == '+' or sign == '-':
        return float(sign + '1')
      return float(sign)
    for line in lines:
      tokens = line.strip().split()
      xt = tokens[0]
      yt = tokens[1] + tokens[2]
      zt = tokens[3] + tokens[4]
      rez = tokens[6]
      x = getCoeff(xt)
      y = getCoeff(yt)
      z = getCoeff(zt)
      M.append([x,y,z])
      V.append(float(rez))
    return M, V



A, B = load_system(pathlib.Path("test.txt"))
print(f"{A=} {B=}")

A=[[2.0, 3.0, -1.0], [1.0, -1.0, 4.0], [3.0, 1.0, 2.0]] B=[5.0, 6.0, 7.0]


### **2. Matrix and Vector Operations (5 points)**

Once you have successfully parsed the matrix and vector, complete the following exercises to manipulate and understand basic matrix and vector operations. Write Python functions for each of these tasks:

#### 2.1. Determinant

Write a function to compute the determinant of matrix $A$. Recall one of the formulae for the determinant of a $3x3$ matrix:
$$ \text{det}(A) = a_{11}(a_{22}a_{33} - a_{23}a_{32}) - a_{12}(a_{21}a_{33} - a_{23}a_{31}) + a_{13}(a_{21}a_{32} - a_{22}a_{31}) $$

In [ ]:
def determinant(m: list[list[float]]) -> float:
    a11, a12, a13 = m[0]
    a21, a22, a23 = m[1]
    a31, a32, a33 = m[2]
    det = a11*(a22*a33-a23*a32)-a12*(a21*a33-a23*a31)+a13*(a21*a32-a22*a31)
    return det

print(f"{determinant(A)=}")

determinant(A)=14.0


#### 2.2. Trace

Compute the sum of the elements along the main diagonal of matrix $A$. For a matrix $A$, this is:
$$ \text{Trace}(A) = a_{11} + a_{22} + a_{33} $$

In [ ]:
def trace(matrix: list[list[float]]) -> float:
  return sum(matrix[i][i] for i in range(3))

print(f"{trace(A)=}")

trace(A)=3.0


#### 2.3. Vector norm

Compute the Euclidean norm of vector $B$, which is:
$$ ||B|| = \sqrt{b_1^2 + b_2^2 + b_3^2} $$

In [ ]:
def norm(vector: list[float]) -> float:
    return sum(x*x for x in vector)**(1/2)

print(f"{norm(B)=}")

norm(B)=10.488088481701515


#### 2.4. Transpose of matrix

Write a function to compute the transpose of matrix $A$. The transpose of a matrix $A$ is obtained by swapping its rows and columns.
    

In [ ]:
def transpose(matrix: list[list[float]]) -> list[list[float]]:
   return [[row[i] for row in matrix] for i in range(len(matrix[0]))]


print(f"{transpose(A)=}")

transpose(A)=[[2.0, 1.0, 3.0], [3.0, -1.0, 1.0], [-1.0, 4.0, 2.0]]


#### 2.5. Matrix-vector multiplication

Write a function that multiplies matrix $A$ with vector $B$.

In [ ]:
def multiply(matrix: list[list[float]], vector: list[float]) -> list[float]:
  return ([[val*vector[0] for val in row]for row in matrix] if len(vector) == 1 else [sum([row[i]*vector[i] for i in range(len(row))])for row in matrix])

print(f"{multiply(A, B)=}")

multiply(A, B)=[21.0, 27.0, 35.0]


### **3. Solving using Cramer's Rule (1 point)**

Now that you have explored basic matrix operations, solve the system of linear equations using Cramer's rule.

**Cramer's Rule:**

Cramer's rule allows you to solve for each unknown $x$, $y$, and $z$ using determinants. For example:
$$ x = \frac{\text{det}(A_x)}{\text{det}(A)}, \quad y = \frac{\text{det}(A_y)}{\text{det}(A)}, \quad z = \frac{\text{det}(A_z)}{\text{det}(A)} $$
where $A_x$, $A_y$, and $A_z$ are matrices formed by replacing the respective column of matrix $A$ with vector $B$.

In [ ]:
def solve_cramer(matrix: list[list[float]], vector: list[float]) -> list[float]:
  return [[determinant([[(vector[idx] if column==i else row[column]) for column in range(len(matrix[0]))]for idx,row in enumerate(matrix)])/determinant(matrix)] for i in range(3)]
print(f"{solve_cramer(A, B)=}")

solve_cramer(A, B)=[[0.35714285714285715], [2.0714285714285716], [1.9285714285714286]]


### **4. Solving using Inversion (3 points)**

Finally, solve the system by computing the inverse of matrix $A$ and multiplying it by vector $B$.
$$ A \cdot X = B \rightarrow X = A^{-1} \cdot B $$
**Adjugate Method for Matrix Inversion:**

To find the inverse of matrix $ A $, you can use the adjugate method:
$$ A^{-1} = \frac{1}{\text{det}(A)} \times \text{adj}(A) $$
where $\text{adj}(A)$ is the adjugate (or adjoint) matrix, which is the transpose of the cofactor matrix of $ A $.

**Cofactor Matrix:**

The cofactor matrix is a matrix where each element is replaced by its cofactor. The cofactor of an element $a_{ij}$ is given by:
$$ (-1)^{i+j} \times \text{det}(M_{ij}) $$
where $M_{ij}$ is the minor of element $a_{ij}$, which is the matrix obtained by removing the $i$-th row and $j$-th column from matrix $A$.

In [ ]:
def minor(matrix: list[list[float]], i: int, j: int) -> list[list[float]]:
  return [[row[y] for y in range(len(row)) if y!=j]for x,row in enumerate(matrix) if x!=i]

def cofactor(matrix: list[list[float]]) -> list[list[float]]:
  return [[((-1)**(i+j))*((lambda m: m[0][0]*m[1][1] - m[0][1]*m[1][0])(minor(matrix,i,j))) for j in range(len(row))]for i,row in enumerate(matrix)]

def adjoint(matrix: list[list[float]]) -> list[list[float]]:
  return transpose(cofactor(matrix))

def solve(matrix: list[list[float]], vector: list[float]) -> list[float]:
  return multiply( multiply( adjoint(matrix), [(1/determinant(matrix))] ), vector)

print(f"{solve(A, B)=}")

solve(A, B)=[0.35714285714285765, 2.071428571428571, 1.9285714285714293]
